In [1]:
from src.vector_store.vector_storage import VectorStorage
import pandas as pd

df = pd.read_csv('qa.csv')
vec_db = VectorStorage()

[2024-05-12 21:30:41,495 _client.py->_send_single_request():1026]INFO: HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
[2024-05-12 21:30:41,546 _client.py->_send_single_request():1026]INFO: HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
[2024-05-12 21:30:41,641 _client.py->_send_single_request():1026]INFO: HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"


In [2]:
import time

for i, row in df.iterrows():
    if row['content_type_id'] == 2:
        vec = vec_db.vector_query_event(row['question'])
        key = vec_db.keyword_query_event(row['question'])
        hybrid = vec_db.hybrid_query_event(row['question'])
    else:
        vec = vec_db.vector_query_base(row['question'])
        key = vec_db.keyword_query_base(row['question'])
        hybrid = vec_db.hybrid_query_base(row['question'])

    print(vec)
    print(key)
    print(hybrid)

    found_vec = row['header'] in str(vec)
    found_key = row['header'] in str(key)
    found_hybrid = row['header'] in str(hybrid)

    print(f'{row["question"]} - {row["header"]} - {found_vec} - {found_key} - {found_hybrid}')

    df.at[i, 'found_vec'] = found_vec
    df.at[i, 'found_key'] = found_key
    df.at[i, 'found_hybrid'] = found_hybrid
    time.sleep(35)


[Object(uuid=_WeaviateUUIDInt('4f13da53-0538-45f0-b166-8ceb97297909'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=0.33970820903778076, certainty=0.8301458954811096, score=0.0, explain_score='', is_consistent=None, rerank_score=None), properties={'text': '“An excellent fusion of top-quality meals and original drinks, ideal for a dinner with your partner or friends." Element combines first-rate food and cocktails. Behind its globally inspired menu from local ingredients stands the well-travelled head chef Tomaš Reger, who has gained culinary experience from around the world. His food is complemented by drinks from champion bartender Jan Liška.', 'date_fetched': datetime.datetime(2024, 5, 12, 20, 37, tzinfo=datetime.timezone.utc), 'header': 'Element Restaurant: Culinary Experience in Brno', 'record_type': 'place', 'url': 'https://www.gotobrno.cz/wp-content/uploads/2023/10/GB23_pruvodce-A6_EN_dvoustrany_web.pdf', 'address': 'Běhounská 7, Brno, Czech Republi

In [3]:
df.to_csv('qa_eval.csv', index=False)

In [9]:
all_vec = df['found_vec'].sum()
all_key = df['found_key'].sum()
all_hybrid = df['found_hybrid'].sum()

all_vec_event = df[df['content_type_id'] == 2]['found_vec'].sum()
all_key_event = df[df['content_type_id'] == 2]['found_key'].sum()
all_hybrid_event = df[df['content_type_id'] == 2]['found_hybrid'].sum()

all_vec_base = df[df['content_type_id'] == 1]['found_vec'].sum()
all_key_base = df[df['content_type_id'] == 1]['found_key'].sum()
all_hybrid_base = df[df['content_type_id'] == 1]['found_hybrid'].sum()

all_vec_event_percentage = all_vec_event / df[df['content_type_id'] == 2].shape[0]
all_key_event_percentage = all_key_event / df[df['content_type_id'] == 2].shape[0]
all_hybrid_event_percentage = all_hybrid_event / df[df['content_type_id'] == 2].shape[0]

all_vec_base_percentage = all_vec_base / df[df['content_type_id'] == 1].shape[0]
all_key_base_percentage = all_key_base / df[df['content_type_id'] == 1].shape[0]
all_hybrid_base_percentage = all_hybrid_base / df[df['content_type_id'] == 1].shape[0]

eval_df = pd.DataFrame({'vector': [all_vec, all_vec_event_percentage, all_vec_base_percentage],
                        'keyword': [all_key, all_key_event_percentage, all_key_base_percentage],
                        'hybrid': [all_hybrid, all_hybrid_event_percentage, all_hybrid_base_percentage]},
                       index=['Celkem', 'EventSchema', 'BaseSchema'])

eval_df.to_csv('qa_eval_summary.csv')

display(eval_df)


,vector,keyword,hybrid
Celkem,82.000000,87.000000,95.000000
EventSchema,0.920000,0.640000,0.840000
BaseSchema,0.776316,0.934211,0.973684
